<a href="https://colab.research.google.com/github/deny-joefakri/Unique-Face-Scrub/blob/main/ResNet50_Unique_Faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Lib


In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
tf.random.set_seed(1234)

In [ ]:
image_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [ ]:
from google.colab import drive  
drive.mount('/content/drive')  

Mounted at /content/drive


In [ ]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/Dataset/face/face_dataset/train/'

val_dir = '/content/drive/MyDrive/Colab Notebooks/Dataset/face/face_dataset/val/'

test_dir = '/content/drive/MyDrive/Colab Notebooks/Dataset/face/face_dataset/test/'

In [ ]:
batch_size = 32
input_shape = (224, 224, 3)

# Preprocess the images


training_set = image_generator.flow_from_directory(batch_size=batch_size,
                                                   directory=train_dir,
                                                   shuffle=True,
                                                   target_size=input_shape[:2],
                                                   class_mode="categorical")

validation_set = image_generator.flow_from_directory(batch_size=batch_size,
                                                   directory=val_dir,
                                                   shuffle=False,
                                                   target_size=input_shape[:2],
                                                   class_mode="categorical")

testing_set = image_generator.flow_from_directory(batch_size=batch_size,
                                                   directory=test_dir,
                                                   shuffle=False,
                                                   target_size=input_shape[:2],
                                                   class_mode="categorical")


Found 3523 images belonging to 50 classes.
Found 250 images belonging to 50 classes.
Found 250 images belonging to 50 classes.


In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from keras.callbacks import ReduceLROnPlateau


def build_model(resnet):
    input_data = Input(input_shape)
    x = resnet(input_data)
    x = GlobalMaxPooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(128,activation='relu')(x) 
    output = Dense(50, activation='softmax')(x)
    
    resnet_model = Model(inputs=input_data, outputs=output)
    
    resnet_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return resnet_model



resnet = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape = input_shape)
resnet.trainable = False
model = build_model(resnet)
model.summary()

callbacks = [ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=0.00001)]
history = model.fit(training_set, epochs=5, 
                         verbose = 1,
                         batch_size=32,
                         callbacks=callbacks,
                         validation_data=validation_set)
# history = model.fit(trainX, trainY,
#                          epochs = 15,
#                          verbose = 1,
#                          batch_size=32,
#                          validation_data = (valX, valY),
#                          callbacks=callbacks)

94765736/94765736 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                              

UnknownError: ignored

In [ ]:
def GetXY(gen):
    listX = []
    listY = []
    for i in range(gen.__len__()):
        gennext = gen.next()
        listX.append(gennext[0])
        listY.append(gennext[1])
    x=np.concatenate(listX)
    y=np.concatenate(listY)
    return (x,y)
    
trainX,trainY = GetXY(training_set)
valX,valY = GetXY(validation_set)
testX,testY = GetXY(testing_set)

In [ ]:
training_set.samples

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10, 5))
plt.grid(True)
plt.show()

In [ ]:
model.evaluate(testing_set)

In [ ]:
test_pred = np.argmax(model.predict(testing_set), axis=1)

In [ ]:
test_loss, test_acc = model.evaluate(testing_set, steps=len(testing_set), verbose=1)
print('Loss: %.3f' % (test_loss * 100.0))
print('Accuracy: %.3f' % (test_acc * 100.0))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay


y_val = testing_set.classes
y_pred = np.argmax(model.predict(testing_set),axis=1)
print(classification_report(y_val,y_pred))

cm = confusion_matrix(y_val, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

# print(classification_report(testing_set,test_pred))
# print(confusion_matrix(testY, predict))



# print(classification_report(testing_set,test_pred))